In [30]:
# setup
data_path = "../data/"
traces_path = "5_ode.csv"

import pandas as pd
df = pd.read_csv(data_path+traces_path)
print(df.head(5))
# print(df['New'])

   case ID resource     activity name   resource+activity     Sitter      New  \
0        1        A        greeting()        A|greeting()   A|hello*    Hello   
1        1        U    question(data)    U|question(data)  U|request    Query   
2        1        A    list(keywords)    A|list(keywords)    A|offer  Request   
3        1        U     set(keywords)     U|set(keywords)  U|request    Query   
4        1        A  prompt(keywords)  A|prompt(keywords)    A|offer  Request   

     New.1  
0    Hello  
1    Query  
2  Request  
3    Query  
4  Request  


In [29]:
# collect dialogues
case = 0
dialogues = []
for index, utterance in df.iterrows():
    if utterance['case ID'] != case:
        dialogues.append('')
        case = utterance['case ID']
    elif utterance['New'] != 'Hello':
        dialogues[-1] += utterance['New'][0]
print(len(dialogues))
print(dialogues)

26
['QRQRQRQRQAF', 'QRFAQRQRQAF', 'QRQRQAF', 'AQRQRQRQAF', 'RQRQRQAQRQRF', 'RQRQAFRFRQRQAQRQRFRFRQAFAFA', 'QRQAQRQRFRF', 'QRQRQAQAFRFAQRFRF', 'RQAQRQAFRQAQRQA', 'QRQRQRQRQRFRQRFRFRFRQRQRQRF', 'RFRFRQRFRQAFRFRQRFRFRQRQRQRQRQRQRQA', 'QAFRF', 'QRQRQRQRQRFRFRFRFRQRQRFAQRFRF', 'QRQAQAFAFAFRFAFA', 'RQRFRQRFRFA', 'QRQRQAFRF', 'AQAAF', 'QRFRFRFRQRFRFRFRQRFRQAFRQRQAFA', 'QRQAQRFAFA', 'QRQRQAQAF', 'RQAQAF', 'RQAQRQAFRFRFRFAF', 'QRQRQRFRQAF', 'RQRQAFRQAQAFAQRFRQRFAFAFA', 'QAQAQAFRFAF', 'RFAFA']
